**Exercise 9.2** In Section 9.3, we simulated the null hypothesis by permutation; that is, we treated the observed values as if they represented the entire population, and randomly assigned the members of the population to the two groups.

An alternative is to use the sample to estimate the distribution for the population, then draw a random sample from that distribution. This process is called resampling. There are several ways to implement resampling, but one of the simplest is to draw a sample with replacement from the observed values, as in Section 9.10.

Write a class named DiffMeansResample that inherits from DiffMeansPermute and overrides RunModel to implement resampling, rather than permutation.

Use this model to test the differences in pregnancy length and birth weight. How much does the model affect the results?

**IMPORTANT!** [ThinkStats2](https://github.com/AllenDowney/ThinkStats2) should be cloned ../.. relative to this dir.

In [1]:
import sys
import os
import math

import numpy as np
import pandas as pd

In [2]:
import thinkstats2_path
import first, thinkstats2, thinkplot

In [3]:
import first

live, firsts, others = first.MakeFrames()
prglngth_data = firsts.prglngth.values, others.prglngth.values
weight_data = firsts.totalwgt_lb.dropna().values, others.totalwgt_lb.dropna().values

In [4]:
# DiffMeansPermute from the book:
class DiffMeansPermute(thinkstats2.HypothesisTest):

    def TestStatistic(self, data):
        group1, group2 = data
        test_stat = abs(group1.mean() - group2.mean())
        return test_stat

    def MakeModel(self):
        group1, group2 = self.data
        self.n, self.m = len(group1), len(group2)
        self.pool = np.hstack((group1, group2))

    def RunModel(self):
        np.random.shuffle(self.pool)
        data = self.pool[:self.n], self.pool[self.n:]
        return data

In [5]:
# Override RunModel for resampling:
class DiffMeansResampling(DiffMeansPermute):
    
    def RunModel(self):
        group1, group2 = self.pool[:self.n], self.pool[self.n:]
        resample1 = np.random.choice(self.pool, self.n, replace=True)
        resample2 = np.random.choice(self.pool, self.m, replace=True)
        return resample1, resample2

In [6]:
iters = 10000

In [7]:
# Permutation test; also record the actual statistics since it doesn't 
# depend upon the simulation:
ht = DiffMeansPermute(prglngth_data)
prglngth_p_val_perm = ht.PValue(iters=iters)
prglngth_stat = ht.actual
ht = DiffMeansPermute(weight_data)
weight_p_val_perm = ht.PValue(iters=iters)
weight_stat = ht.actual

In [8]:
# Resampling test:
ht = DiffMeansResampling(prglngth_data)
prglngth_p_val_res = ht.PValue(iters=iters)
ht = DiffMeansResampling(weight_data)
weight_p_val_res = ht.PValue(iters=iters)

In [10]:
print("Preg Length Actual Stat={:.6f}".format(prglngth_stat))
print("Birth Weight Actual Stat={:.6f}".format(weight_stat))

print("Comparison Table:")
df = pd.DataFrame(
    [
        [prglngth_p_val_perm, weight_p_val_perm],
        [prglngth_p_val_res, weight_p_val_res]
    ],
    columns=['Preg Length p-val', 'Birth Weight p-val'],
    index=['Permutation', 'Resampling']
)
df.index.name = 'Simulation'
print(df)

Preg Length Actual Stat=0.078037
Birth Weight Actual Stat=0.124761
Comparison Table:
             Preg Length p-val  Birth Weight p-val
Simulation                                        
Permutation             0.1654                 0.0
Resampling              0.1696                 0.0


**Conclusion:** The p-values returned by both simulation methods are very close to each other and they lead to the same interpretation:
 - pregnancy length difference is statistically insignificant
 - birth weight length is statistically significant